In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import time
import sklearn
import warnings
import pickle
import catboost
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)


warnings.filterwarnings(action="ignore", category=FutureWarning)

pd.__version__, sklearn.__version__, catboost.__version__

('2.1.1', '1.3.1', '1.2.2')

In [2]:
SCRATCH_DIR = "/scratch/siads696f23_class_root/siads696f23_class/psollars"

# For local dev
# SCRATCH_DIR = "./../data"

In [3]:
consolidated_df = pd.read_parquet(
    f"{SCRATCH_DIR}/top_airline_airport_consolidated_features_2019.parquet"
)

print(consolidated_df.shape)
# (2626181, 36)

consolidated_df.head()

(2626181, 36)


,Quarter,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,DistanceGroup,year_of_manufacture,...,aircraft_type,aircraft_manufacturer,aircraft_model,DepDel15,ArrDel15,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,1,11,5,0940,1110,90,421,2,2016,...,5,AIRBUS,A320-214,0,0,0,0,0,0,0
1,1,1,11,5,1810,1945,95,421,2,2007,...,5,AIRBUS,A320-214,1,1,0,0,0,0,29
2,1,1,11,5,1915,2150,275,1770,8,2011,...,5,AIRBUS,A320-214,0,0,0,0,0,0,0
3,1,1,11,5,1915,2150,275,1770,8,2011,...,5,AIRBUS,A320-214,0,0,0,0,0,0,0
4,1,1,11,5,1420,1640,140,763,4,2011,...,5,AIRBUS,A320-214,0,0,0,0,0,0,0


In [4]:
num_cols = [
    "Quarter",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "CRSElapsedTime",
    "Distance",
    "DistanceGroup",
    "year_of_manufacture",
    "num_seats",
    "Origin_LATITUDE",
    "Origin_LONGITUDE",
    "Dest_LATITUDE",
    "Dest_LONGITUDE",
]

cat_cols = [
    "Reporting_Airline",
    "Tail_Number",
    "Origin",
    "Dest",
    "company_type",
    "registrant",
    "aircraft_usage",
    "engine_type",
    "registration_status",
    "engine_manufacturer",
    "engine_model",
    "aircraft_type",
    "aircraft_manufacturer",
    "aircraft_model",
]

delay_cols = [
    "DepDel15",
    "ArrDel15",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay",
]

In [5]:
df = consolidated_df.copy()

# Choose preferred delay type and convert the label to "delayed"
df["delayed"] = (df["DepDel15"].eq(1)) | (df["ArrDel15"].eq(1))
df = df.drop(delay_cols, axis="columns")

df = df.reset_index(drop=True)

print(df.value_counts("delayed"))
# delayed
# False    2006414
# True      619767

print(df.shape)
# (2626181, 30)

df.head()

delayed
False    2006414
True      619767
Name: count, dtype: int64
(2626181, 30)


,Quarter,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,DistanceGroup,year_of_manufacture,...,registrant,aircraft_usage,engine_type,registration_status,engine_manufacturer,engine_model,aircraft_type,aircraft_manufacturer,aircraft_model,delayed
0,1,1,11,5,0940,1110,90,421,2,2016,...,other,1T,5,V,CFM,CFM56-5B4/3,5,AIRBUS,A320-214,False
1,1,1,11,5,1810,1945,95,421,2,2007,...,WELLS FARGO TRUST CO NA TRUSTEE,1T,5,V,CFM,CFM56-5B4/P,5,AIRBUS,A320-214,True
2,1,1,11,5,1915,2150,275,1770,8,2011,...,BANK OF UTAH TRUSTEE,1T,5,V,CFM,CFM56-5B4,5,AIRBUS,A320-214,False
3,1,1,11,5,1915,2150,275,1770,8,2011,...,BANK OF UTAH TRUSTEE,1T,5,V,CFM,CFM56-5B4,5,AIRBUS,A320-214,False
4,1,1,11,5,1420,1640,140,763,4,2011,...,WELLS FARGO TRUST CO NA TRUSTEE,1T,5,V,CFM,CFM56-5B4,5,AIRBUS,A320-214,False


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split


# CatBoost is converting True/False into string values for some reason
# casting the labels as 1/0 ints here
X = df.drop("delayed", axis=1)
y = df["delayed"].astype(int)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OrdinalEncoder(), cat_cols),
    ]
)

X_transformed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
from catboost import CatBoostClassifier, Pool


def fit_catboost_classifier(learning_rate=0.6, depth=6, l2_leaf_reg=1):
    start_time = time.time()

    model = CatBoostClassifier(
        eval_metric="Logloss",
        auto_class_weights="Balanced",
        verbose=250,
        random_state=42,
        random_strength=1,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        early_stopping_rounds=20,
    )

    model.fit(X_train, y_train, eval_set=Pool(X_test, y_test))

    end_time = time.time()

    print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

    return model

In [8]:
def build_evaluation_metrics(model_eval, title="CatBoost"):
    y_pred = model_eval.predict(X_test)

    metrics_scores = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC AUC": roc_auc_score(y_test, y_pred, average="macro", multi_class="ovr"),
        "Log Loss": model_eval.best_score_["learn"]["Logloss"]
    }

    metrics_df = pd.DataFrame(list(metrics_scores.items()), columns=["Metric", "Score"])

    base = alt.Chart(metrics_df).encode(
        y=alt.Y(
            "Metric:N", axis=alt.Axis(title="Metric"), sort=metrics_df["Metric"].values
        ),
        x=alt.X("Score:Q", axis=alt.Axis(title="Score"), scale=alt.Scale(domain=[0, 1])),
    )

    bar = base.mark_bar().encode(
        color=alt.Color("Metric:N", legend=None), text=alt.Text("Score:Q", format=".2f")
    )

    text = base.mark_text(
        align="right",
        baseline="middle",
        color="white",
        dx=-10,
    ).encode(text=alt.Text("Score:Q", format=".2f"))

    chart = (
        (bar + text)
        .properties(title=f"{title} Evaluation Metrics", width=600, height=300)
        .configure_axis(labelFontSize=12, titleFontSize=14)
    )

    chart.save(f"visualizations/{title}_metrics.png")

    return metrics_scores

In [9]:
def build_sensitivity_analysis_visualization(df, parameter=""):
    data_melted = df.melt(
        id_vars=[f"{parameter}"],
        value_vars=["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "Log Loss"],
        var_name="metric",
        value_name="value",
    )

    base = alt.Chart(data_melted).encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Value"), scale=alt.Scale(domain=[0, 1])),
        x=alt.X("metric:N", axis=alt.Axis(title="Metric"), sort=df.columns.values),
    )

    bar = (
        base.mark_bar(width=25)
        .encode(
            tooltip=[f"{parameter}", "metric", "value"],
            color=alt.Color("metric:N", legend=None),
        )
        .properties(width=180)
    )

    text = base.mark_text(
        align="center",
        color="white",
        size=10,
        dy=10,
    ).encode(text=alt.Text("value:Q", format=".2f"))

    chart = (bar + text).facet(column=alt.Column(f"{parameter}:N", sort=df[parameter].values))

    return chart

In [10]:
learning_rates = [0.01, 0.1 , 0.6, 1]

learning_rate_df  = pd.DataFrame()

for i, lr in enumerate(learning_rates):
    m = fit_catboost_classifier(learning_rate=lr)

    metrics = build_evaluation_metrics(m, f"CatBoost - Learning Rate:{lr}")

    metrics["Learning Rate"] = lr

    learning_rate_df = pd.concat((learning_rate_df, pd.DataFrame(metrics, index=[i])))

learning_rate_df

TBB Warning: The number of workers is currently limited to 7. The request for 35 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	learn: 0.6925716	test: 0.6925931	best: 0.6925931 (0)	total: 168ms	remaining: 2m 47s
250:	learn: 0.6567504	test: 0.6569810	best: 0.6569810 (250)	total: 26.9s	remaining: 1m 20s
500:	learn: 0.6485474	test: 0.6489120	best: 0.6489120 (500)	total: 51.9s	remaining: 51.7s
750:	learn: 0.6434594	test: 0.6439014	best: 0.6439014 (750)	total: 1m 17s	remaining: 25.7s
999:	learn: 0.6399745	test: 0.6405143	best: 0.6405143 (999)	total: 1m 42s	remaining: 0us

bestTest = 0.6405142848
bestIteration = 999

Elapsed time: 118.6234 seconds
0:	learn: 0.6877897	test: 0.6878691	best: 0.6878691 (0)	total: 160ms	remaining: 2m 39s
250:	learn: 0.6274537	test: 0.6285624	best: 0.6285624 (250)	total: 26.3s	remaining: 1m 18s
500:	learn: 0.6181506	test: 0.6203202	best: 0.6203202 (500)	total: 52.6s	remaining: 52.4s
750:	learn: 0.6123676	test: 0.6154860	best: 0.6154860 (750)	total: 1m 19s	remaining: 26.3s
999:	learn: 0.6080206	test: 0.6119847	best: 0.6119847 (999)	total: 1m 45s	remaining: 0us

bestTest = 0.6119846512
be

,Accuracy,Precision,Recall,F1 Score,ROC AUC,Log Loss,Learning Rate
0,0.619122,0.341293,0.660100,0.449948,0.633282,0.639974,0.01
1,0.662509,0.378266,0.668183,0.483064,0.664469,0.608021,0.10
2,0.679112,0.393891,0.667651,0.495471,0.675152,0.579300,0.60
3,0.674379,0.388403,0.660874,0.489261,0.669713,0.588241,1.00


In [11]:
with open(f"{SCRATCH_DIR}/sensitivity_analysis_learning_rate_df.pkl", "wb") as f:
    pickle.dump(learning_rate_df, f)


In [12]:
chart = build_sensitivity_analysis_visualization(learning_rate_df, parameter="Learning Rate")

chart.save(f"visualizations/learning_rate_sensitivity_metrics.png")

chart

alt.FacetChart(...)

In [13]:
depths = [4, 6, 8, 10]

depth_df = pd.DataFrame()

for i, parameter in enumerate(depths):
    m = fit_catboost_classifier(depth=parameter)

    metrics = build_evaluation_metrics(m, f"CatBoost - Depth:{lr}")

    metrics["Depth"] = parameter

    depth_df = pd.concat((depth_df, pd.DataFrame(metrics, index=[i])))

depth_df

0:	learn: 0.6739934	test: 0.6741472	best: 0.6741472 (0)	total: 109ms	remaining: 1m 48s
250:	learn: 0.6204410	test: 0.6226761	best: 0.6226761 (250)	total: 21.9s	remaining: 1m 5s
500:	learn: 0.6122680	test: 0.6163155	best: 0.6163155 (500)	total: 43.4s	remaining: 43.2s
750:	learn: 0.6075028	test: 0.6131816	best: 0.6131816 (750)	total: 1m 5s	remaining: 21.8s
999:	learn: 0.6036041	test: 0.6109176	best: 0.6109176 (999)	total: 1m 27s	remaining: 0us

bestTest = 0.6109176453
bestIteration = 999

Elapsed time: 102.9835 seconds
0:	learn: 0.6700107	test: 0.6701612	best: 0.6701612 (0)	total: 137ms	remaining: 2m 17s
250:	learn: 0.6046655	test: 0.6107080	best: 0.6107080 (250)	total: 27.4s	remaining: 1m 21s
500:	learn: 0.5929966	test: 0.6050157	best: 0.6049847 (494)	total: 54.3s	remaining: 54.1s
750:	learn: 0.5842543	test: 0.6019983	best: 0.6019983 (750)	total: 1m 20s	remaining: 26.5s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6011177443
bestIteration = 904

Shrink model to fi

,Accuracy,Precision,Recall,F1 Score,ROC AUC,Log Loss,Depth
0,0.664719,0.379726,0.664117,0.483181,0.664511,0.603604,4
1,0.679112,0.393891,0.667651,0.495471,0.675152,0.579300,6
2,0.683345,0.397786,0.665053,0.497815,0.677024,0.569486,8
3,0.682271,0.396373,0.662359,0.495954,0.675390,0.561570,10


In [14]:
with open(f"{SCRATCH_DIR}/sensitivity_analysis_depth_df.pkl", "wb") as f:
    pickle.dump(depth_df, f)


In [15]:
chart = build_sensitivity_analysis_visualization(depth_df, parameter="Depth")

chart.save(f"visualizations/depth_sensitivity_metrics.png")

chart

alt.FacetChart(...)

In [16]:
l2 = [1, 3, 5]

l2_df = pd.DataFrame()

for i, parameter in enumerate(l2):
    m = fit_catboost_classifier(l2_leaf_reg=parameter)

    metrics = build_evaluation_metrics(m, f"CatBoost - L2 Leaf Regularization:{lr}")

    metrics["L2 Leaf Regularization"] = parameter

    l2_df = pd.concat((l2_df, pd.DataFrame(metrics, index=[i])))

l2_df

0:	learn: 0.6700107	test: 0.6701612	best: 0.6701612 (0)	total: 119ms	remaining: 1m 58s
250:	learn: 0.6046655	test: 0.6107080	best: 0.6107080 (250)	total: 31.2s	remaining: 1m 33s
500:	learn: 0.5929966	test: 0.6050157	best: 0.6049847 (494)	total: 1m 2s	remaining: 1m 2s
750:	learn: 0.5842543	test: 0.6019983	best: 0.6019983 (750)	total: 1m 31s	remaining: 30.3s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6011177443
bestIteration = 904

Shrink model to first 905 iterations.
Elapsed time: 127.3809 seconds
0:	learn: 0.6700107	test: 0.6701612	best: 0.6701612 (0)	total: 143ms	remaining: 2m 23s
250:	learn: 0.6052729	test: 0.6110638	best: 0.6110638 (250)	total: 28.5s	remaining: 1m 24s
500:	learn: 0.5927479	test: 0.6041148	best: 0.6041027 (499)	total: 57.2s	remaining: 57s
750:	learn: 0.5840694	test: 0.6011519	best: 0.6011519 (750)	total: 1m 25s	remaining: 28.4s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6008431721
bestIteration = 797

Shrink model to

,Accuracy,Precision,Recall,F1 Score,ROC AUC,Log Loss,L2 Leaf Regularization
0,0.679112,0.393891,0.667651,0.495471,0.675152,0.579300,1
1,0.679238,0.393964,0.667248,0.495418,0.675095,0.582166,3
2,0.681054,0.395588,0.665844,0.496311,0.675798,0.577326,5


In [17]:
with open(f"{SCRATCH_DIR}/sensitivity_analysis_l2_df.pkl", "wb") as f:
    pickle.dump(l2_df, f)


In [19]:
chart = build_sensitivity_analysis_visualization(l2_df, parameter="L2 Leaf Regularization")

chart.save(f"visualizations/l2_sensitivity_metrics.png")

chart

alt.FacetChart(...)

In [20]:
def select_features(df, features=[]):
    select_num_cols = [f for f in num_cols if f in features]
    select_cat_cols = [f for f in cat_cols if f in features]

    # Choose preferred delay type and convert the label to "delayed"
    df["delayed"] = (df["DepDel15"].eq(1)) | (df["ArrDel15"].eq(1))
    df = df.drop(delay_cols, axis="columns")
    df = df.reset_index(drop=True)

    # select only given features
    df = df[features + ["delayed"]]

    # CatBoost is converting True/False into string values for some reason
    # casting the labels as 1/0 ints here
    X = df.drop("delayed", axis=1)
    y = df["delayed"].astype(int)

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), select_num_cols),
            ("cat", OrdinalEncoder(), select_cat_cols),
        ]
    )

    X_transformed = preprocessor.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_transformed, y, test_size=0.2, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test

In [25]:
def fit_catboost_select_features(X_train, X_test, y_train, y_test):
    start_time = time.time()

    model = CatBoostClassifier(
        eval_metric="Logloss",
        auto_class_weights="Balanced",
        verbose=250,
        random_state=42,
        random_strength=1,
        learning_rate=0.6,
        depth=6,
        l2_leaf_reg=1,
        early_stopping_rounds=20,
    )

    model.fit(X_train, y_train, eval_set=Pool(X_test, y_test))

    end_time = time.time()

    print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

    return model

In [22]:
# feature importance from catboost_grid_search

feature_importance = {
    "DayofMonth": 18.7318,
    "Month": 17.7656,
    "DayOfWeek": 8.4516,
    "CRSDepTime": 7.8024,
    "Origin_LONGITUDE": 5.8846,
    "Reporting_Airline": 5.2053,
    "Dest_LONGITUDE": 4.8329,
    "CRSArrTime": 4.3172,
    "Origin_LATITUDE": 3.6464,
    "Quarter": 2.9810,
    "CRSElapsedTime": 2.7767,
    "Dest_LATITUDE": 2.4248,
    "Origin": 2.4247,
    "Distance": 2.3765,
    "Dest": 2.0555,
    "Tail_Number": 1.9868,
    "aircraft_model": 1.5336,
    "year_of_manufacture": 1.4117,
    "engine_model": 1.1736,
    "registrant": 0.9765,
    "num_seats": 0.3142,
    "engine_manufacturer": 0.2106,
    "aircraft_manufacturer": 0.1770,
    "aircraft_type": 0.1556,
    "DistanceGroup": 0.1319,
    "registration_status": 0.0704,
    "engine_type": 0.0669,
    "company_type": 0.0573,
    "aircraft_usage": 0.0568,
}

In [24]:
importance_values = [0, 2, 3, 5]

feature_importance_df = pd.DataFrame()

for i, value in enumerate(importance_values):
    selected_features = [f for f, v in feature_importance.items() if v > value]
    top_num_features = len(selected_features)

    X_train, X_test, y_train, y_test = select_features(consolidated_df, selected_features)

    m = fit_catboost_select_features(X_train, X_test, y_train, y_test)

    metrics = build_evaluation_metrics(m, f"CatBoost - Top {top_num_features} Features")

    metrics["Top Features"] = top_num_features

    feature_importance_df = pd.concat((feature_importance_df, pd.DataFrame(metrics, index=[i])))

feature_importance_df

0:	learn: 0.6700107	test: 0.6701612	best: 0.6701612 (0)	total: 161ms	remaining: 2m 41s
100:	learn: 0.6175204	test: 0.6203731	best: 0.6203731 (100)	total: 12s	remaining: 1m 46s
200:	learn: 0.6077780	test: 0.6126114	best: 0.6126114 (200)	total: 23.6s	remaining: 1m 33s
300:	learn: 0.6017052	test: 0.6089162	best: 0.6089162 (300)	total: 34.5s	remaining: 1m 20s
400:	learn: 0.5972333	test: 0.6068314	best: 0.6068314 (400)	total: 45.1s	remaining: 1m 7s
500:	learn: 0.5929966	test: 0.6050157	best: 0.6049847 (494)	total: 55.6s	remaining: 55.3s
600:	learn: 0.5894019	test: 0.6039270	best: 0.6039270 (600)	total: 1m 5s	remaining: 43.8s
700:	learn: 0.5859400	test: 0.6026214	best: 0.6026020 (694)	total: 1m 16s	remaining: 32.6s
800:	learn: 0.5828412	test: 0.6017588	best: 0.6017588 (800)	total: 1m 26s	remaining: 21.5s
900:	learn: 0.5799030	test: 0.6011623	best: 0.6011228 (896)	total: 1m 36s	remaining: 10.6s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6011177443
bestIteration = 904


,Accuracy,Precision,Recall,F1 Score,ROC AUC,Log Loss,Top Features
0,0.679112,0.393891,0.667651,0.495471,0.675152,0.579300,29
1,0.681668,0.396321,0.666828,0.497161,0.676540,0.580987,15
2,0.682469,0.396910,0.665094,0.497140,0.676465,0.582585,9
3,0.667120,0.380522,0.653743,0.481044,0.662497,0.599988,6


In [27]:
with open(f"{SCRATCH_DIR}/sensitivity_analysis_features_df.pkl", "wb") as f:
    pickle.dump(feature_importance_df, f)


In [28]:
chart = build_sensitivity_analysis_visualization(feature_importance_df, "Top Features")

chart.save(f"visualizations/feature_sensitivity_metrics.png")

chart

alt.FacetChart(...)

In [ ]:
# !pip install vl-convert-python